In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm
from Model import *
from sklearn.metrics import classification_report
import tensorflow_addons as tfa
from keras.utils import io_utils
from pandarallel import pandarallel
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
pandarallel.initialize(progress_bar=True, nb_workers=16)
tqdm.pandas()

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [2]:
train = pd.read_csv("../../dataset/goodreads_train.csv")
test = pd.read_csv("../../dataset/goodreads_test.csv")
vocabulary = np.load('../../vocabulaires/voc_without_std_word_count_5.npy', allow_pickle=True)
#train['review_text'] = train['review_text'].str.replace('[^\w\s]','')
train = shuffle(train)

In [3]:
def convert_timestamp(x):
    import pandas as pd
    if pd.isna(x):  # parallel_apply
        return 0.0
    else:
        try:
            return float(pd.Timestamp(x).value / 10**18)
        except:
            return 0
train[['read_at','date_added','date_updated' ,'started_at']] = train[['read_at','date_added','date_updated' ,'started_at']].parallel_applymap(convert_timestamp)

In [4]:
inputs_data = train[['review_text','n_comments', 'n_votes','read_at','date_added','date_updated','started_at']]
outputs_data = keras.utils.to_categorical(train['rating'], num_classes=6)
train_in, validation_in, train_out, validation_out = train_test_split(inputs_data, outputs_data, test_size=0.2)

In [5]:
train_in = [train_in['review_text'], train_in['n_comments'], train_in['n_votes'], train_in['read_at'], train_in['date_added'], train_in['date_updated'], train_in['started_at']]
validation_in = [validation_in['review_text'], validation_in['n_comments'], validation_in['n_votes'], validation_in['read_at'], validation_in['date_added'], validation_in['date_updated'], validation_in['started_at']]

In [6]:
model_list = [transf_test]
kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4)
bias_regularizer=regularizers.L2(1e-4)
activity_regularizer=regularizers.L2(1e-5)
params=[
    #{"dropout_rate": .0,"kernel_regularizer": None, "bias_regularizer": None, "activity_regularizer": None},
    {"dropout_rate": .2,"kernel_regularizer": None, "bias_regularizer": None, "activity_regularizer": None},
    #{"dropout_rate": .0,"kernel_regularizer": regularizers.L1L2(l1=1e-5, l2=1e-4),
    #  "bias_regularizer": regularizers.L2(1e-4),
    #  "activity_regularizer": regularizers.L2(1e-5)},
    # {"dropout_rate": .2,"kernel_regularizer": regularizers.L1L2(l1=1e-5, l2=1e-4),
    #  "bias_regularizer": regularizers.L2(1e-4),
    #  "activity_regularizer": regularizers.L2(1e-5)}
]

def scheduler(epoch, lr):
    if epoch < 1:
        return lr
    else:
        return lr * tf.math.exp(-0.2)
seeds = [42]
for seed in seeds:
    keras.utils.set_random_seed(seed)
    for model_obj in model_list:
        for param in params:

            dropout_rate = param['dropout_rate']
            kernel_regularizer = param['kernel_regularizer']
            bias_regularizer = param['bias_regularizer']
            activity_regularizer = param['activity_regularizer']

            if kernel_regularizer is None:
                regularizers_ = "None"
            else:
                regularizers_ = "L1L2"
            if dropout_rate == .0:
                dropout = "0"
            else:
                dropout = f"{dropout_rate}"

            model = model_obj.model(vocabulary, dropout_rate, kernel_regularizer, bias_regularizer, activity_regularizer)
            model.compile(optimizer=keras.optimizers.Adamax(learning_rate=0.003),
                           loss=keras.losses.categorical_crossentropy,
                           metrics=[keras.metrics.categorical_accuracy, tfa.metrics.F1Score(num_classes=6, average='weighted')]
                           )
            print(model.name)

            if not os.path.exists(f"checkpoint/{model.name}/"):
                os.mkdir(f"checkpoint/{model.name}")
            if not os.path.exists(f"checkpoint/{model.name}/{seed}/"):
                os.mkdir(f"checkpoint/{model.name}/{seed}/")
            if not os.path.exists(f"checkpoint/{model.name}/{seed}/{dropout}"):
                os.mkdir(f"checkpoint/{model.name}/{seed}/{dropout}")
            if not os.path.exists(f"checkpoint/{model.name}/{seed}/{dropout}/{regularizers_}"):
                os.mkdir(f"checkpoint/{model.name}/{seed}/{dropout}/{regularizers_}")

            if not os.path.exists(f"logs/{model.name}/"):
                os.mkdir(f"logs/{model.name}")
            if not os.path.exists(f"logs/{model.name}/{seed}/"):
                os.mkdir(f"logs/{model.name}/{seed}/")
            if not os.path.exists(f"logs/{model.name}/{seed}/{dropout}"):
                os.mkdir(f"logs/{model.name}/{seed}/{dropout}")
            if not os.path.exists(f"logs/{model.name}/{seed}/{dropout}/{regularizers_}"):
                os.mkdir(f"logs/{model.name}/{seed}/{dropout}/{regularizers_}")

            chekpoint = keras.callbacks.ModelCheckpoint(f'checkpoint/{model.name}/{seed}/{dropout}/{regularizers_}/model.h5',
            monitor='val_f1_score', mode='max', save_best_only=True)
            stop_nan = keras.callbacks.TerminateOnNaN()

            sheduler = keras.callbacks.LearningRateScheduler(scheduler,0)
            tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=f"logs/{model.name}/{seed}/{dropout}/{regularizers_}/")
            model.fit(x=train_in, y=train_out, validation_data=(validation_in, validation_out), batch_size=128, epochs=15, callbacks=[sheduler, stop_nan, chekpoint, tensorboard_callback])

trans_test
Epoch 1/15
5625/5625 [==============================] - 808s 144ms/step - loss: 1.1131 - categorical_accuracy: 0.5213 - f1_score: 0.5113 - val_loss: 1.0461 - val_categorical_accuracy: 0.5464 - val_f1_score: 0.5359 - lr: 0.0025
Epoch 3/15
5625/5625 [==============================] - 808s 144ms/step - loss: 1.0696 - categorical_accuracy: 0.5407 - f1_score: 0.5332 - val_loss: 1.0361 - val_categorical_accuracy: 0.5518 - val_f1_score: 0.5383 - lr: 0.0020
Epoch 4/15
5625/5625 [==============================] - 817s 145ms/step - loss: 1.0423 - categorical_accuracy: 0.5533 - f1_score: 0.5466 - val_loss: 1.0316 - val_categorical_accuracy: 0.5570 - val_f1_score: 0.5579 - lr: 0.0016
Epoch 5/15
5625/5625 [==============================] - 812s 144ms/step - loss: 0.9852 - categorical_accuracy: 0.5794 - f1_score: 0.5745 - val_loss: 1.0152 - val_categorical_accuracy: 0.5684 - val_f1_score: 0.5624 - lr: 9.0358e-04
Epoch 8/15
5625/5625 [==============================] - 814s 145ms/step - los

In [9]:
res = model.predict(train_in)

22500/22500 [==============================] - 157s 7ms/step


In [10]:
val_res = model.predict(validation_in)

5625/5625 [==============================] - 39s 7ms/step


In [13]:
val_out = np.argmax(validation_out, axis=1)
train_out = np.argmax(train_out, axis=1)


In [14]:
test[['read_at','date_added','date_updated' ,'started_at']] = test[['read_at','date_added','date_updated' ,'started_at']].parallel_applymap(convert_timestamp)

test_data = [test['review_text'], test['n_comments'], test['n_votes'], test['read_at'], test['date_added'], test['date_updated'], test['started_at']]
restest = model.model.predict(test_data)

14939/14939 [==============================] - 109s 7ms/step


In [15]:
ff = []
for line in tqdm(val_res):
    tmp = -2
    category = None
    for i in (range(6)):
        if line[i] > tmp:
            category = i
            tmp = line[i]
    ff.append(category)
val_data = np.array(ff)

ff = []
for line in tqdm(restest):
    tmp = -2
    category = None
    for i in (range(6)):
        if line[i] > tmp:
            category = i
            tmp = line[i]
    ff.append(category)
test_data = np.array(ff)

ff = []
for line in tqdm(res):
    tmp = -2
    category = None
    for i in (range(6)):
        if line[i] > tmp:
            category = i
            tmp = line[i]
    ff.append(category)
train_data = np.array(ff)

  0%|          | 0/180000 [00:00<?, ?it/s]

  0%|          | 0/478033 [00:00<?, ?it/s]

  0%|          | 0/720000 [00:00<?, ?it/s]

In [16]:
print(classification_report(train_out, train_data))

              precision    recall  f1-score   support

           0       0.73      0.44      0.55     24800
           1       0.55      0.34      0.42     22993
           2       0.55      0.52      0.53     58041
           3       0.64      0.65      0.64    151064
           4       0.63      0.72      0.67    250900
           5       0.76      0.70      0.73    212202

    accuracy                           0.66    720000
   macro avg       0.64      0.56      0.59    720000
weighted avg       0.66      0.66      0.66    720000



In [17]:
print(classification_report(val_out, val_data))

              precision    recall  f1-score   support

           0       0.66      0.40      0.49      6188
           1       0.51      0.31      0.38      5725
           2       0.50      0.47      0.48     14586
           3       0.58      0.59      0.58     37908
           4       0.58      0.68      0.63     62788
           5       0.72      0.66      0.69     52805

    accuracy                           0.61    180000
   macro avg       0.59      0.51      0.54    180000
weighted avg       0.62      0.61      0.61    180000



In [18]:
test['rating'] = test_data

id = test['review_id'].to_numpy()
rating = test['rating'].to_numpy()
df = pd.DataFrame( columns=['review_id', 'rating'])
df['review_id'] = id
df['rating'] = rating

In [19]:
df.to_csv('submission_unet5_embedding_class_weights_model.csv',index=False)

In [16]:
model.model.save('unet5')

INFO:tensorflow:Assets written to: unet5\assets


INFO:tensorflow:Assets written to: unet5\assets
